# 심층 신경망

확실히 로지스틱 회귀보다 성능이 좋고, 인공 신경망은 층을 많이 추가할 수 있으니까 추가해서 성능을 더 높여보자.

In [1]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

## 2개의 층

다시 케라스 API를 이용하여 패션 MSINT 데이터셋을 불러오자.  

In [3]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

그다음 이미지의 픽셀값을 0~255 범위에서 0~1 사이로 변환하고, 28 * 28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼침. 마지막으로 사이킷런의 train_test_split() 함수로 훈련 세트와 검증 세트로 나뉨. 이것은 1절에서 했던 것과 같음.

In [4]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

이제 인공 신경망 모델에 층을 2개 추가해보자.  
1절을 만든 신경망 모델과 다른 점은 입력층과 출력층 사이에 밀집층이 추가된 것. 이렇게 입력층과 출력층 사이에 있는 모든 층을 **은닉충**이라고 부름.  
은닉층에는 주황색 원으로 활성화 함수가 표시되어 있음.  
활성화 함수는 신경망 층의 선형 방적식의 계산값에 적용하는 함수.  
출력층에 적용하는 활성화 함수는 종류가 제한되어 있음.  
이진 분류일 경우 시그모이드 함수를 사용하고 다중 분류일 경우 소프트맥스 함수를 사용.

회귀를 위한 신경망의 출력층에서는 어떤 활성화 함수를 사용?  
분류 문제는 클래스에 대한 확률을 출력하기 위해 활성화 함수를 사용. 회귀의 출력은 임의의 어떤 숫자이므로 활성화 함수를 적용할 필요가 없음.  
즉 출력층의 선형 방정식의 계산을 그대로 출력. 이렇게 하려면 Dense 층의 activation 매개변수에 아무런 값을 지정하지 않음.  
그런데 은닉층에 왜 활성화 함수를 적용?  
은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없는 셈이므로 선형 계산을 적당하게 비선형적으로 비틀어 주어야 함.  
그래야 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 할 수 있다.  

다른 책에서는 인공 시경망 그림에 활성화 함수가 없던데..?  
인공 신경망을 그림으로 나타낼 때 활성화 함수를 생략하는 경우가 많은데 이는 절편과 마찬가지로 번거로움을 피하기 위해서 활성화 함수를 별개의 층으로 생각하지 않고 층에 포함되어 있다고 간주하기 때문. 그림에서 보이지는 않아도 모든 신경망의 은닉층에는 항상 활성화 함수가 있다.

많이 사용하는 활성화 함수는 시그모이드 함수.  
이 함수는 뉴런의 출력 z값을 0과 1 사이로 압축.  
시그모이드 활성화 함수를 사용한 은닉충과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어 보겠다.  
이전 절에서 언급했듯이 케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 꼭 지정해주어야 함.

In [5]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

dense1이 은닉층이고 100개의 뉴런을 가진 밀집층임.  
활성화 함수를 'sigmoid'로 지정했고 input_shape 매개변수에서 입력의 크기를 (784.)로 지정했다.  
은닉충의 뉴런 개수를 정하는 데는 특별한 기준이 없다. 몇 개의 뉴런을 두어야 할지 판단하기 위해서는 상당한 경험이 필요.  
여기에서 한가지 제약 사항이 있다면 적어도 출력층의 뉴런보다는 많게 만들어야 한다.  
클래스 10개에 대한 확률을 예측해야 하는데 이전 은닉층의뉴런이 10개보다 적다면 부족한 정보가 전달될 것이다.  
그다음 dense2는 출력층. 10개의 클래스를 분류하므로 10개의 뉴런을 두었고 활성화 함수는 소프트맥스 함수로 지정.

## 심층 신경망 만들기

이제 앞에서 만든 dense1과 dense2 객체를 Sequential 클래스에 추가하여 심층 신경망을 만들어 보자.

In [6]:
model = keras.Sequential([dense1, dense2])

Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 이와 같이 dense1과 dense2를 리스트를 만들어 전달.  
여기서 주의할 것은 출력층을 가장 마지막에 두어야 한다는 것.  
이 리스트는 가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 나열해야 함.  
인공신경망의 강력한 성능은 바로 이렇게 층을 추가하여 입력 데이터에 대해 연속적인 학습을 진행하는 능력에서 나온다.

케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


맨 첫 줄에 모델의 이름이 나옴. 그다음 이 모델에 들어 있는 층이 순서대로 나열된다. 이 순서는 맨 처음 추가한 은닉층에서 출력츠으이 순서대로 나열된다.  
층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력됨.  
층을 만들 때 name 매개변수로 이름을 지정할 수 있다. 층 이름을 지정하지 않으면 케라스가 자동으로 'dense'라고 이름을 붙임.  
출력 크기를 보면 (None, 100)이다. 첫 번째 차원은 샘플의 개수를 나타냄. 샘플 개수가 아직 정의되어 있지 않기 때문에 None. 왜?  
케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행. 바로 미니 배치 경사 하강법 이용.

케라스의 기본 미니 배치 크기는 32개. 이 값은 fit() 메서드에서 batch_size 매개벼수로 바꿀 수 있다. 따라서 샘플 개수를 고정하지 앟고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정. 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 배치 차원이라고 부름.  
두 번째 100은 쉬움. 은닉충의 뉴런 개수를 100개로 두었으니 100개의 출력이 나올 것임.  
즉 샘플마다 784개의 픽셀값이 은닉충을 통과하면서 100개의 특성으로 압축되었음.  
마지막으로 모델 파라미터 개수가 출력됨. 이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있음. 그리고 뉴런마다 1개의 절편이 있음.  

두 번째 층의 출력 크기는 (None, 10)이다. 배치 차원은 동일하게 None이고 출력 뉴런 개수가 10개이기 때문.  
이 층의 모델 파라미터 개수는 몇 개일까요?  
100개의 은닉층 뉴런과 10개의 출력층 뉴런이 모두 연결되고 출력층의 뉴런마다 하나의 절편이 있기 때문에 총 1010개의 모델 파라미터가 있음.  
summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79510개로 나옴.  
은닉층과 출력층의 파라미터 개수를 합친 값. 그 아래 훈련되지 않는 파라미터는 0으로 나옴.  
간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있음.  이런 층의 파라미터 개수가 여기에 나타남.  

## 층을 추가하는 다른 방법
모델을 훈련하기 전에 Seqeuntial 클래스에 층을 추가하는 다른 방법을 알아보자.  
Dense 클래스의 객체 dense1, dense2를 만ㄷ르어 Sequential 클래스에 전달. 이 두 객체를 따로 저장하여 쓸 일이 없기 때문에 다음처럼 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많음.

In [8]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

이렇게 작업하면 추가되는 층을 한눈에 쉽게 알아보는 장점이 있다. 이전과 달리 이번에는 Sequential 클래스의 name 매개변수로 모델의 이름을 지정. 또 Dense 층의 name 매개변수에 층의 이름을 hidden과 output으로 각각 지정. 모델의 이름과 달리 층의 이름은 반드시 영문이어야 함.  
summary() 메서드의 출력에 이름이 잘 반영되는가 확인.

In [9]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


2개의 Dense 층이 이전과 동일하게 추가되었고 파라미터 개수도 같다. 바뀐 것은 모델 이름과 층 이름. 여러 모델과 많은 층을 사용할 때 name 매개변수를 사용하면 구분하기 쉬움.  
  
이 방법이 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어진다.  
또 조건에 따라 층을 추가할 수도 없다.  
Sequential 클래스에서 층을 추가할 때 가장 널리 사용하는 방법은 add() 메서드이다.  
이 방법은 다음처럼 Sequential 클래스의 객체를 만들고 이 객체의 add() 메서드를 호출하여 층을 추가.

In [10]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

여기에서도 Dense 클래스의 객체를 따로 변수에 담지 않고 바로 add() 메서드로 전달.  
이 방법은 한눈에 추가되는 층을 볼 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가할 수 있다.  
summary() 메서드의 결과에서 층과 파라미터 개수는 당연히 동일.  

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이제 모델을 훈련해 보자.  
compile() 메서드의 설정은 1절에서 했던 것과 동일. 여기에서도 5번의 에포크 동안 훈련.

In [12]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 32s 20ms/step - loss: 0.5652 - accuracy: 0.8082
Epoch 2/5
1500/1500 [==============================] - 32s 21ms/step - loss: 0.4096 - accuracy: 0.8521
Epoch 3/5
1500/1500 [==============================] - 29s 20ms/step - loss: 0.3743 - accuracy: 0.8658
Epoch 4/5
1500/1500 [==============================] - 30s 20ms/step - loss: 0.3507 - accuracy: 0.8735
Epoch 5/5
1500/1500 [==============================] - 31s 20ms/step - loss: 0.3327 - accuracy: 0.8784


훈련 세트에 대한 성능을 보면 추가된 층이 성능을 향상시켰다는 것을 잘 알 수 있음.  
인공 신경망에 몇 개의 층을 추가하더라도 compile() 메서드와 fit() 메서드의 사용법은 동일하다.  
이것이 케라스 API의 장점. 필요하면 여러 개의 층을 추가하고 실험.

다음에는 이미지 분류 문제에서 높은 성능을 낼 수 있는 활성화 함수에 대해 알아보자.

## 렐루 활성화 함수
초창기 인공 신경망의 은닉층에 많이 사용된 활성화 함수는 시그모이드 함수였지만 이 함수에 단점이 존재.  
이 함수의 오른쪽과 왼쪽 끝을 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못함.  
특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게 만듬.  
이를 개선하기 위해 다른 종류의 활성화 함수가 제안됨. 바로 **렐루**함수.  
  
렐루 함수는 아주 간단. 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수일 경우 0으로 만듬.  
렐루 함수는 max(0,z)와 같이 쓸 수 있다. 이 함수는 z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력.  
렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져 있음.  
은닉층의 활성화 함수에 시그모이드 함수 대신 렐루 함수를 적용하기 전에 케라스에서 제공하는 편리한 층 하나를 더 살펴보자.

패션 MNIST 데이터는 28 * 28 크기이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape() 메서드를 사용해 1차원으로 펼쳤음.  
직접 이렇게 1차원으로 펼쳐도 좋지만 케라스에서는 이를 위한 Flatten 층을 제공함.  
사실 Flatten 클래스는 배치 자원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다.  
입력에 곱해지는 가중치나 절편이 없다. 따라서 인공 신경망의 성능을 위해 기여하는 바는 없다.  
하지만 Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라 부른다.  
Flatten 층은 다음 코드처럼 입력층 바로 뒤에 추가.

In [13]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

첫 번째 Dense 층에 있던 input_shape 매개변수를 Flatten 층으로 옮김. 또 첫 번째 Dense 층의 활성화 함수를 'relu'로 바꿔보자.  
이 신경망을 깊이가 3인 신경망이라고 부르진 않는다. Flatten 클래스는 학습하는 층이 아니므로. 모델의 summary() 메서드를 호출해보면 이런 점을 더 확실히 알 수 있다.  

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


첫 번째 등장하는 Flatten 클래스에 포함된 모델 파라미터는 0개.  
케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 장점이 있음.  
앞의 출력에서 784개의 입력이 첫 번째 은닉층에 전달된다는 것을 알 수 있는데 이는 이전에 만들었던 모델에서는 쉽게 눈치채기 어려웠음.  
입력 데이터에 대한 전처리 과정을 가능한 모델에 포함시키는 것이 케라스 API의 철학 중 하나.  
케라스 API는 입력 데이터에 대한 전처리 과정을 될 수 있으면 모델에 포함한다.  
  
훈련 데이터를 다시 준비해서 모델을 훈련해보면 아래와 같다. 이 절의 서두에 있던 코드와 동일하지만 reshape() 메서드를 적용하지 않았다.

In [15]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

모델을 컴파일하고 훈련하는 것은 다음 코드처럼 이전과 동일하다.

In [16]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 32s 20ms/step - loss: 0.5262 - accuracy: 0.8136
Epoch 2/5
1500/1500 [==============================] - 30s 20ms/step - loss: 0.3908 - accuracy: 0.8579
Epoch 3/5
1500/1500 [==============================] - 27s 18ms/step - loss: 0.3551 - accuracy: 0.8724
Epoch 4/5
1500/1500 [==============================] - 30s 20ms/step - loss: 0.3342 - accuracy: 0.8814
Epoch 5/5
1500/1500 [==============================] - 30s 20ms/step - loss: 0.3170 - accuracy: 0.8861


시그모이드 함수를 사용했을 때와 비교하면 성능이 조금 향상되었음. 크지 않지만 렐루 함수의 효과를 보았음.  
검증 세트에서의 성능도 확인해보자.

In [17]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 7s 19ms/step - loss: 0.3884 - accuracy: 0.8694


[0.3884405195713043, 0.8694166541099548]

1절의 은닉층을 추가하지 않은 경우보다 몇 퍼센트 성능이 향상되었다. 지금까지는 모델을 5번의 에포크 동안 훈련하였음.  
이보다 더 훈련하지 않을 이유가 없다. 그전에 인공 신경망의 하이퍼파라미터에 대해 잠시 알아보고 이번 절을 마무리.

## 옵티마이저
3장에서 하이퍼파라미터는 모델이 학습하지 않아 사람이 지정해주어야 하는 파라미터라고 설명.  
지금까지 다룬 하이퍼파라미터는 추가할 은닉층의 개수, 뉴런 개수, 활성화 함수, 층의 종류, 배치 사이즈 매개벼수, 에포크 매개변수 등이 있다.

이번 절에서는 은닉층을 하나 추가. 하지만 여러 개의 은닉층을 추가할 수도 있다.  
추가할 은닉충의 개수는 모델이 학습하는 것이 아니라 우리가 지정해 주어야 할 하이퍼파라미터임.  
그럼 은닉층의 뉴런 개수도 하이퍼파라미터일까요? 맞음.  
또 활성화 함수도 선택해야 할 하이퍼파라미터 중 하나. 심지어 층의 종류도 하이퍼파라미터임.  
이 장에서는 가장 기본적인 밀집층만 다루지만, 다른 종류의 층을 선택할 수도 있다.

케라스는 기본적으로 미니배치 경사 하강법을 사용하여 미니배치 개수는 32개이다.  
fit() 메서드의 batch_size 매개변수에서 이를 조정할 수 있고 이 역시 하이퍼파라미터이다.  
또한 fit() 메서드의 epochs 매개변수도 하이퍼파라미터임. 반복 회수에 따라 다른 모델이 만들어진다.  
마지막으로 compile() 메서드에서는 케라스의 기본 경사 하강법 알고리즘인 RMSprop을 사용.  
케라스는 다양한 종류의 경사 하강법 알고리즘을 제공. **옵티마이저**라고 함.  
역시 다른 옵티마이저를 테스트하지 않을 이유는 없다. 또한 RMSprop의 학습률 또한 조정할 하이퍼 파라미터 중 하나임.

처음부터 모델을 구상하고 각종 하이퍼파라미터의 최적값을 찾는 것은 어려운 작엄.  
가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD.  
이름이 SGD이지만 1개의 샘플을 뽑아서 훈련하지 않고 앞서 언급한 것처럼 기본적으로 미니배치를 사용.  
SGD 옵티마이저를 사용하려면 compile() 메서드의 optimizer 매개변수를 'sgd'로 지정.

In [18]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

이 옵티마이저는 tensorflow.keras.optimizers 패키지 아래 SGD 클래스로 구현되어 있음.  
'sgd'라는 문자열은 이 클래스의 기본 설정 매개변수로 생성한 객체와 동일. 즉 다음 코든ㄴ 위의 코드와 정확히 동일함.

In [19]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

sgd와 'sgd'의 차이는?  
sgd = keras.optimizers.SGD()처럼 SGD 클래스 객체를 만들어 사용해야 하는데,  
번거로움을 피하고자 'sgd'라 지정하면 자동으로 SGD 클래스 객체를 만들어 줌. 

만약 SGD 클래스의 학습률 기본값이 0.01일 때 이를 바꾸고 싶다면 다음과 같이 원하는 학습률을 learning_rate 매개변숭 지정하여 사용.

In [20]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공. SGD 클래스의 momentum 매개변수의 기본값은 0이다.  
이를 0보다 큰 값으로 지정하면 마치 이전의 그레이디어트를 가속도처럼 사용하는 **모멘텀 최적화**를 사용.  
보통 momentum 매개변수는 0.9 이상을 지정.

SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 **네스테로프 모멘텀 최적화**를 사용.

In [21]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현.  
대부분의 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공.  
모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있음.  
이렇게 하면 안정적으로 최적점에 수렴할 가능성이 있음 이런 학습률을 **적응적 학습률**이라고 하고  
이러한 방식들은 학습률 매개변수를 튜닝하는 수고를 덜 수 있는 것이 장점.

적응적 학습률을 사용하는 대표적인 옵티마이저는 Adagrad와 RmSprop이다.  
각각 compile() 메서드의 optimizer 매개변수에 'adagrad;와 'rmprop'으로 지정할 수 있음.  
optimizer 매개변수의 기본값이 바로 'rmsprop'이다.  
이 두 옵티마이저의 매개변수를 바꾸고 싶다면 SGD와 같이 Adagrad와 RmSprop 클래스 객체를 만들어 사용하면 된다.

In [22]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

RMSprop도 마찬가지

In [22]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam이다. Adam은 RMSprop과 함께 맨처음 시도해 볼 수 있는 좋은 알고리즘임.  
Adam 클래스도 keras.optimizers 패키지 아래에 있다.  
적응적 학습률을 사용하는 이 3개의 클래스는 learning_rate 매개변수의 깁노값으로 모두 0.001을 사용.

Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련해보자.  
먼저 모델을 다시 생성.

In [23]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

compile() 메서드의 optimizer를 'adam'으로 설정하고 5번의 에포크 동안 훈련.

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 36s 23ms/step - loss: 0.5257 - accuracy: 0.8154
Epoch 2/5
1500/1500 [==============================] - 32s 22ms/step - loss: 0.3947 - accuracy: 0.8585
Epoch 3/5
1500/1500 [==============================] - 32s 21ms/step - loss: 0.3534 - accuracy: 0.8709
Epoch 4/5
1500/1500 [==============================] - 36s 24ms/step - loss: 0.3276 - accuracy: 0.8791
Epoch 5/5
1500/1500 [==============================] - 34s 23ms/step - loss: 0.3074 - accuracy: 0.8874


이 출력 결과를 보면 기본 RMSprop을 사용했을 때와 거의 같은 성능을 보여 준다.  
마지막으로 검증 세트에서의 성능도 확인.

In [25]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 9s 23ms/step - loss: 0.3463 - accuracy: 0.8763


[0.34633150696754456, 0.8763333559036255]

환경마다 조금씩 차이가 있을 수 있지만 여기서는 기본 RMSprop보다 조금 나은 성능을 낸다.

# 정리
여러 개의 층을 추가하여 다층 인공 신경망을 만드는 방법을 배움.  
특별히 이런 인공 신경망을 심층 신경망이라고 부름. 케라스 API를 사용하여 층을 추가하는 여러 가지 방법을 배움.  
케라스 모델의 정보를 요약해 주는 summary() 메서드. 출력값의 의미를 이해하고 모델 파라미터 개수를 계산해 맞추어 봄.  
모델 파라미터 개수를 계산하는 과정은 모델을 올바르게 이해하고 있는지 확인하는 좋은 방법 중 하나.  
은닉층에 적용한 시그모이드 활성화 함수 대신에 새로운 렐루 활성화 함수에 대해 배웠고 이를 적용해 약간의 성능을 향상시킴.  
또한 다양한 고급 경사 하강법 옵티마이저들을 적용하는 방법도 살펴봄. 케라스 API를 사용하면 이런 작업이 어렵지 않고 직관적으로 구성할 수 있음.  